In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'leapgestrecog:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F39466%2F61155%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240704%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240704T172320Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da1222b2b873b9f52add41a25f3227d0f85bb790a10be122f82fdc704fdf958836f3b6f8845af8d8779d105a9147cd787341f492f797f5c1004a7b7a06a03dc36d75170b47885edbf62973b3becd6236d4c9a629101e786e506b02c35b07e2979058c71262421e2cc12085b866741f9527aa1cc798a36388da023d40c78979088760d9b3252f1266e3a1cd85bb8a0552d61ef3b8774ab5cc54fae9a2f30d30f9c5e6722ca358229b1fe0baa3925faf22c6c210a02b5cde69d9000b6a6782c4d47273c9936c970f6518e701a177a08077f3bb8698ee61701ca62e64b360459a3b686f11eb5c2a69762d4d17982b4bb28e853723882dcc4f6485d9ba522ddf88f09'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2286085038 bytes downloaded
Downloaded and uncompressed: leapgestrecog
Data source import complete.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
DATASET_PATH = "/kaggle/input/leapgestrecog/leapGestRecog"
batch_size = 32
image_size = (128, 128)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting 20% for validation
)

In [ ]:
generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify 'training' or 'validation' for either subset
)

Found 16000 images belonging to 10 classes.


In [ ]:
validation_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 4000 images belonging to 10 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 30

In [ ]:
history = model.fit(
    generator,

    epochs=epochs,
    validation_data=validation_generator,
)

Epoch 1/30
500/500 [==============================] - 146s 276ms/step - loss: 0.7071 - accuracy: 0.7482 - val_loss: 1.4939 - val_accuracy: 0.6833
Epoch 2/30
500/500 [==============================] - 124s 248ms/step - loss: 0.1085 - accuracy: 0.9582 - val_loss: 2.1458 - val_accuracy: 0.6173
Epoch 3/30
500/500 [==============================] - 113s 227ms/step - loss: 0.0569 - accuracy: 0.9774 - val_loss: 1.9155 - val_accuracy: 0.7380
Epoch 4/30
500/500 [==============================] - 114s 228ms/step - loss: 0.0572 - accuracy: 0.9749 - val_loss: 1.7064 - val_accuracy: 0.6970
Epoch 5/30
500/500 [==============================] - 133s 265ms/step - loss: 0.0412 - accuracy: 0.9814 - val_loss: 2.0572 - val_accuracy: 0.7255
Epoch 6/30
500/500 [==============================] - 114s 228ms/step - loss: 0.0408 - accuracy: 0.9800 - val_loss: 1.5819 - val_accuracy: 0.7283
Epoch 7/30
500/500 [==============================] - 114s 228ms/step - loss: 0.0346 - accuracy: 0.9835 - val_loss: 1.5383 -

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nValidation accuracy:', test_acc)

125/125 - 22s - loss: 0.9889 - accuracy: 0.8530 - 22s/epoch - 178ms/step

Validation accuracy: 0.8529999852180481


In [ ]:
from sklearn.metrics import classification_report
# Load the test dataset
X_test, y_test = [], []
for i in range(len(validation_generator)):
    batch_X, batch_y = validation_generator[i]
    X_test.append(batch_X)
    y_test.append(batch_y)
X_test = np.concatenate(X_test)
y_test = np.concatenate(y_test)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert the one-hot encoded predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded true labels to class labels
y_true_classes = np.argmax(y_test, axis=1)

# Create a classification report
report = classification_report(y_true_classes, y_pred_classes)

print(report)

125/125 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

           0       0.86      0.97      0.92       400
           1       1.00      1.00      1.00       400
           2       0.96      0.99      0.97       400
           3       0.99      1.00      0.99       400
           4       0.68      0.57      0.62       400
           5       0.93      1.00      0.96       400
           6       0.63      0.94      0.75       400
           7       0.90      0.77      0.83       400
           8       0.87      0.28      0.42       400
           9       0.79      0.99      0.88       400

    accuracy                           0.85      4000
   macro avg       0.86      0.85      0.83      4000
weighted avg       0.86      0.85      0.83      4000

